This code will generate the feature vector, using sub grids to calculate the overlap

In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')
from os.path import exists


##  Step 0. Throw out fires that have 0 viirs detects on any day. (fires are already filtered to be in the right lat/lon range

In [2]:
# load in the polygons and attributes
fire_polygons = gpd.read_file('fire_polygons_only.geojson')
fire_attributes = pd.read_csv('fire_polygons_attributes.csv')
incidents = pd.read_csv('unique_fires.csv')
print(len(fire_polygons), len(fire_attributes), len(incidents))

#throw out fires that have 0 days with detects
daily_drop_inds = []
incident_drop_inds = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    if len(pts[pts==0])==len(pts):
        #print(item, 'oops all zeros')
        daily_drop_inds = np.append(daily_drop_inds, np.where(fire_attributes['Incident Number']==item)[0])
        incident_drop_inds = np.append(incident_drop_inds, np.where(incidents['Incident Number']==item)[0])

daily_drop_inds = [int(p) for p in daily_drop_inds]
#print(daily_drop_inds)

fire_attributes = fire_attributes.drop(labels= daily_drop_inds)
fire_attributes = fire_attributes.drop(labels='Unnamed: 0', axis=1)
fire_polygons = fire_polygons.drop(labels = daily_drop_inds)
incidents = incidents.drop(labels = incident_drop_inds)
print(len(fire_polygons), len(fire_attributes), len(incidents))

fire_attributes


22280 22280 1354
19137 19137 1036


,Incident Number,Fire Name,Current Day,Lat Fire,Lon Fire,Number of VIIRS points
5,10662684.0,PAINTED WAGON,2019-04-08,33.786944,-112.753333,3.0
6,10662684.0,PAINTED WAGON,2019-04-09,33.786944,-112.753333,0.0
7,10662684.0,PAINTED WAGON,2019-04-10,33.786944,-112.753333,0.0
8,10663171.0,LONE MOUNTAIN,2019-05-26,33.808056,-105.738611,0.0
9,10663171.0,LONE MOUNTAIN,2019-05-27,33.808056,-105.738611,0.0
...,...,...,...,...,...,...
22275,11980587.0,CREEK 5,2020-12-25,33.402646,-117.266795,16.0
22276,11980587.0,CREEK 5,2020-12-26,33.402646,-117.266795,0.0
22277,11980587.0,CREEK 5,2020-12-27,33.402646,-117.266795,0.0
22278,11980587.0,CREEK 5,2020-12-28,33.402646,-117.266795,0.0


## 1. Figure out the start and end date we will use for each fire

In [ ]:
# first check what fraction of the fire lifetimes are 0 days
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    #print(len(np.where(pts==0)[0]))
    print('The fire lifetime is ', len(np.where(pts!=0)[0])*100/len(pts), '% nonzero detections')

In [3]:
keep_days = []
for item in incidents['Incident Number']:
    pts = fire_attributes['Number of VIIRS points'].values[fire_attributes['Incident Number']==item]
    dys = fire_attributes['Current Day'].values[fire_attributes['Incident Number']==item]
    nonzero = np.where(pts!=0)[0]
    first_day = dys[nonzero[0]]
    last_day = dys[nonzero[len(nonzero)-1]]
    keep_days = np.append(keep_days, np.where((fire_attributes['Incident Number']==item)&(fire_attributes['Current Day']>=first_day)&(fire_attributes['Current Day']<=last_day))[0])

keep_days = [int(p) for p in keep_days]
print(len(keep_days))
fire_attributes = fire_attributes.iloc[keep_days, :]
fire_polygons = fire_polygons.iloc[keep_days, :]
fire_attributes = fire_attributes.reset_index(drop=True) #reset the indices beecause we dropped some
fire_polygons = fire_polygons.reset_index(drop=True) #reset the indices beecause we dropped some

fire_polygons



8161


,geometry
0,"POLYGON ((-112.75455 33.78632, -112.76012 33.7..."
1,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
2,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
3,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
4,"POLYGON ((-105.73275 33.80235, -105.73861 33.8..."
...,...
8156,"POLYGON ((-112.11669 34.11898, -112.12564 34.1..."
8157,"POLYGON ((-117.03702 33.90632, -117.04484 33.9..."
8158,"POLYGON ((-117.27595 33.37881, -117.27495 33.3..."
8159,"POLYGON ((-117.29549 33.36558, -117.29713 33.4..."


## 2. Loop through the days and get the raw features (temp at different pressure levels, wind, pressure, precip, terrain info) and the labels (QFED PM2.5 from the next day)

In [ ]:
ndays=2
# dictionaries that track the variables, dimensions, times and levels
varis = {'/STAGEIV/ST4.*12.24h.nc':['A_PCP_GDS5_SFC_acc24h'], 
         '/STAGEIV/st4_conus.*12.24h.nc':['APCP_P8_L1_GST0_acc']}
print(varis['/STAGEIV/st4_conus.*12.24h.nc'])
#0=lat, 1=lon, 2=time, 3=level 
dims = {'/STAGEIV/ST4.*12.24h.nc':['g5_lat_0', 'g5_lon_1'], 
        '/STAGEIV/st4_conus.*12.24h.nc':['gridlat_0', 'gridlon_0']}

times = {'TLML':12, 'QLML':12, 'SPEEDLML':12, 'PS':12, 'T':12, 'QV':12, 'biomass':12}
levels = {'T':[500, 700], 'QV':[700]} 
file_suffix = '.nc4'

#pre-allocate the data frame
num_days = len(fire_attributes)
fire_features = gpd.GeoDataFrame({'Incident Number': np.nan*np.zeros(num_days), 'Fire Name': np.nan*np.zeros(num_days), 
                             'Current Day': np.nan*np.zeros(num_days), 'Lat Fire': np.nan*np.zeros(num_days), 
                             'Lon Fire': np.nan*np.zeros(num_days), 'Number of VIIRS points': np.nan*np.zeros(num_days),
                             'A_PCP_GDS5_SFC_acc24h_0':np.nan*np.zeros(num_days),'APCP_P8_L1_GST0_acc_0':np.nan*np.zeros(num_days),
                            'A_PCP_GDS5_SFC_acc24h_1':np.nan*np.zeros(num_days),'APCP_P8_L1_GST0_acc_1':np.nan*np.zeros(num_days),
                            'geometry': np.nan*np.zeros(num_days)}) # pre-allocate the dataframe
#'fccs': np.nan*np.zeros(num_days),'slp': np.nan*np.zeros(num_days),'asp': np.nan*np.zeros(num_days), 
fire_features['Incident Number'] = fire_attributes['Incident Number'].values
fire_features['Fire Name'] = fire_attributes['Fire Name'].values
fire_features['Current Day'] = fire_attributes['Current Day'].values
fire_features['Lat Fire'] = fire_attributes['Lat Fire'].values
fire_features['Lon Fire'] = fire_attributes['Lon Fire'].values
fire_features['Number of VIIRS points'] = fire_attributes['Number of VIIRS points'].values
fire_features['geometry'] = fire_polygons['geometry'].values

#print(fire_features)
unique_days = np.unique(fire_features['Current Day'].values)
#print(np.where(unique_days=='2020-07-5'))
print(len(unique_days))
for dd in range(len(unique_days)):
    for dyf in range(ndays):
        date=np.datetime64(unique_days[dd])+np.array(dyf, 'timedelta64[D]')
        print(date)
        date=str(date)
        yr = date[0:4]
        mo = date[5:7]
        dy = date[8:10]
        print(yr, mo,dy)
        #find the polygons for the given day
        # assign i index based on date i=0 for dates <= 20200727, i = 1 for dates > 20200727
        if np.datetime64(date)<=np.datetime64('2020-07-27'):
            i = '/STAGEIV/ST4.*12.24h.nc'
        else:
            i = '/STAGEIV/st4_conus.*12.24h.nc'
        #get the path to the file
        data_path = '/data2/lthapa/'+yr+i
        if '*' in data_path:
            data_path = data_path.replace('*', yr+mo+dy)
        if np.datetime64(date)>=np.datetime64('2020-09-01') and np.datetime64(date)<=np.datetime64('2020-09-30'):
            data_path = data_path.replace('400', '401')

        #load the file in as a netcdf dataset
        print(data_path)
        dat = nc.Dataset(data_path)

        ndims = len(dims[i])
        lat = dat[dims[i][0]][:]
        lon = dat[dims[i][1]][:]
        #print(lat)
        #print(lat.shape, lon.shape)

        #calculate the areas of the lat/lon grid for the DATASET
        lat_corners, lon_corners = calculate_grid_cell_corners(lat, lon)
        inds = np.where(fire_features['Current Day']==unique_days[dd])[0]
        polys = fire_features['geometry'].iloc[inds].values

        for p in range(len(inds)):
            tic = time.time()
            #add a halo to the polygon
            poly = polys[p]
            poly_buff = poly.buffer(2)
            poly_series = gpd.GeoSeries(poly)
            # get the bounds of the buffered polygon

            w,s,e,n = poly_buff.bounds
            #print(w,s,e,n)
            i0, i1, j0, j1 =  bbox2ij(lon, lat, [w, e, s, n])

            #lat_sub_inds, lon_sub_inds = np.where((lat>=s)&(lat<=n)&(lon>=w)&(lon<=e))
            #print(lat_sub_inds, lon_sub_inds)

            #tic = time.time()
            overlaps = calculate_overlaps(lat,lon,lat_corners,lon_corners,j0,j1,i0,i1, poly) #calculate the overlap between the poly and the lat/lon grid
            print(np.where(overlaps!=0))
            #toc = time.time()
            #print('Time elapsed for calculating overlaps for polygon:', toc-tic)
            for j in varis[i]: #loop over the variable names, which are lists
                var = dat[j][:,:].data
                var[var==1e20] = np.nan
                #print('the dimensions of', j, 'are', var.shape, 'and', j, 'has', ndims, 'dimensions')
                #select the times and levels we need
                var_weighted = np.multiply(var,overlaps)
                var_ave = np.nansum(var_weighted)
                print(var_ave)
                name_in_table = j+'_'+str(dyf)
                fire_features.loc[inds[p],name_in_table] = var_ave

            toc = time.time()
            print('Time to process polygon=',toc-tic)
        dat.close() 
fire_features

['APCP_P8_L1_GST0_acc']
442
2019-02-17
2019 02 17
/data2/lthapa/2019/STAGEIV/ST4.2019021712.24h.nc
(array([258, 259]), array([176, 176]))
0.0
Time to process polygon= 0.7909574508666992
2019-02-18
2019 02 18
/data2/lthapa/2019/STAGEIV/ST4.2019021812.24h.nc
(array([258, 259]), array([176, 176]))
0.0
Time to process polygon= 0.8308413028717041
2019-02-18
2019 02 18
/data2/lthapa/2019/STAGEIV/ST4.2019021812.24h.nc
(array([258, 259]), array([176, 176]))
0.0
Time to process polygon= 0.7975642681121826
2019-02-19
2019 02 19
/data2/lthapa/2019/STAGEIV/ST4.2019021912.24h.nc
(array([258, 259]), array([176, 176]))
0.0
Time to process polygon= 0.8315963745117188
2019-02-24
2019 02 24
/data2/lthapa/2019/STAGEIV/ST4.2019022412.24h.nc
(array([454, 455]), array([411, 411]))
0.0
Time to process polygon= 0.5205152034759521
2019-02-25
2019 02 25
/data2/lthapa/2019/STAGEIV/ST4.2019022512.24h.nc
(array([454, 455]), array([411, 411]))
0.0
Time to process polygon= 0.5205020904541016
2019-03-06
2019 03 06
/d

(array([638, 638]), array([199, 200]))
0.0
Time to process polygon= 0.5761942863464355
(array([349, 350]), array([220, 220]))
9.721666340542304
Time to process polygon= 0.691162109375
2019-05-08
2019 05 08
/data2/lthapa/2019/STAGEIV/ST4.2019050812.24h.nc
(array([556]), array([71]))
0.0
Time to process polygon= 0.7081582546234131
(array([638, 638]), array([199, 200]))
0.0
Time to process polygon= 0.5657958984375
2019-05-09
2019 05 09
/data2/lthapa/2019/STAGEIV/ST4.2019050912.24h.nc
(array([556]), array([71]))
0.0
Time to process polygon= 0.7198705673217773
(array([638, 638]), array([199, 200]))
0.0
Time to process polygon= 0.5647296905517578
2019-05-09
2019 05 09
/data2/lthapa/2019/STAGEIV/ST4.2019050912.24h.nc
(array([556]), array([71]))
0.0
Time to process polygon= 0.6999368667602539
2019-05-10
2019 05 10
/data2/lthapa/2019/STAGEIV/ST4.2019051012.24h.nc
(array([556]), array([71]))
0.0
Time to process polygon= 0.7002279758453369
2019-05-14
2019 05 14
/data2/lthapa/2019/STAGEIV/ST4.2019

(array([268]), array([303]))
0.0
Time to process polygon= 0.6774575710296631
2019-06-02
2019 06 02
/data2/lthapa/2019/STAGEIV/ST4.2019060212.24h.nc
(array([316, 316, 317, 317, 318]), array([255, 256, 255, 256, 255]))
0.0
Time to process polygon= 0.7007267475128174
(array([294]), array([255]))
0.0
Time to process polygon= 0.6987583637237549
(array([268]), array([303]))
0.0
Time to process polygon= 0.6618335247039795
2019-06-03
2019 06 03
/data2/lthapa/2019/STAGEIV/ST4.2019060312.24h.nc
(array([316, 316, 317, 317, 318]), array([255, 256, 255, 256, 255]))
0.0
Time to process polygon= 0.698636531829834
(array([294]), array([255]))
0.0
Time to process polygon= 0.6944210529327393
(array([268]), array([303]))
0.0
Time to process polygon= 0.6611039638519287
2019-06-03
2019 06 03
/data2/lthapa/2019/STAGEIV/ST4.2019060312.24h.nc
(array([315, 316, 316, 317, 317, 318]), array([255, 255, 256, 255, 256, 255]))
0.0
Time to process polygon= 0.6973543167114258
(array([294, 294, 295, 295]), array([255, 

(array([268, 268]), array([302, 303]))
0.0
Time to process polygon= 0.6842124462127686
(array([260]), array([332]))
0.0
Time to process polygon= 0.6551847457885742
(array([255]), array([323]))
0.0
Time to process polygon= 0.6615045070648193
2019-06-09
2019 06 09
/data2/lthapa/2019/STAGEIV/ST4.2019060912.24h.nc
(array([317, 317, 318]), array([255, 256, 255]))
0.0
Time to process polygon= 0.6779520511627197
(array([274]), array([324]))
0.0
Time to process polygon= 0.6335160732269287
(array([293, 293, 294, 294, 295, 295]), array([255, 256, 255, 256, 255, 256]))
0.0
Time to process polygon= 0.699760913848877
(array([411, 411, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459,
       460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472,
       473]), array([56, 57,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       

(array([364, 364, 365, 365, 390, 391, 392, 393, 394, 395, 396, 397, 398,
       399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411,
       412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424,
       425, 426, 427, 428]), array([59, 60, 59, 60,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0]))
0.0
Time to process polygon= 0.8376719951629639
(array([380, 380]), array([396, 397]))
0.0
Time to process polygon= 0.5588316917419434
(array([367]), array([339]))
0.0
Time to process polygon= 0.5659515857696533
(array([263, 263, 264, 264, 265]), array([253, 254, 253, 254, 254]))
0.0
Time to process polygon= 0.7336568832397461
(array([268, 268]), array([302, 303]))
0.0
Time to process polygon= 0.6725494861602783
(array([367, 368]), array([108, 108]))
0.0
Time to process polygon= 0.7710208892822266
2019-06-13
2019 06 13
/data2/lthapa/2019/STAGEIV/ST4.201

...

(array([304, 305]), array([220, 220]))
0.0
Time to process polygon= 0.7150208950042725
(array([381]), array([243]))
0.0
Time to process polygon= 0.6430842876434326
(array([310, 310]), array([243, 244]))
0.0
Time to process polygon= 0.6926829814910889
(array([436, 436, 437, 437]), array([93, 94, 93, 94]))
0.0
Time to process polygon= 0.7800664901733398
(array([594, 594, 595, 595]), array([177, 178, 177, 178]))
0.0
Time to process polygon= 0.5909957885742188
(array([569, 569, 570, 570]), array([217, 218, 217, 218]))
0.0
Time to process polygon= 0.5771088600158691
(array([337]), array([360]))
0.30000001192092896
Time to process polygon= 0.5799775123596191
(array([351, 351, 352, 352, 352, 353, 353, 354]), array([240, 241, 239, 240, 241, 239, 240, 239]))
0.0
Time to process polygon= 0.6992185115814209
(array([347, 347, 348, 348]), array([239, 240, 239, 240]))
0.0
Time to process polygon= 0.6879007816314697
(array([617]), array([234]))
0.0
Time to process polygon= 0.5264072418212891
(array([

(array([514]), array([310]))
0.0
Time to process polygon= 0.5128521919250488
(array([571]), array([372]))
0.0
Time to process polygon= 0.4494760036468506
(array([444, 445]), array([350, 350]))
0.0
Time to process polygon= 0.524644136428833
(array([670]), array([135]))
0.0
Time to process polygon= 0.5765855312347412
(array([683, 683, 684, 684]), array([141, 142, 141, 142]))
0.0
Time to process polygon= 0.573596715927124
(array([378]), array([295]))
0.0
Time to process polygon= 0.6056756973266602
(array([665, 665, 665, 665, 666, 666, 666, 666, 667, 667, 667, 667, 668,
       668, 668, 669, 669, 669]), array([175, 176, 177, 178, 175, 176, 177, 178, 175, 176, 177, 178, 175,
       176, 177, 175, 176, 177]))
0.0
Time to process polygon= 0.6017622947692871
(array([365, 365, 366]), array([110, 111, 111]))
0.0
Time to process polygon= 0.7675778865814209
(array([351, 351, 352]), array([221, 222, 221]))
0.0
Time to process polygon= 0.6926796436309814
(array([599, 599, 600, 600]), array([142, 143

(array([588, 589]), array([231, 231]))
0.0
Time to process polygon= 0.5499331951141357
(array([644, 644]), array([258, 259]))
0.0
Time to process polygon= 0.5092582702636719
(array([431]), array([271]))
0.0
Time to process polygon= 0.591254472732544
(array([571]), array([372]))
0.0
Time to process polygon= 0.44791293144226074
(array([670]), array([135]))
0.0
Time to process polygon= 0.5774736404418945
(array([350]), array([481]))
3.0978660153088415
Time to process polygon= 0.6047863960266113
(array([683, 683, 684, 684]), array([141, 142, 141, 142]))
0.0
Time to process polygon= 0.5731375217437744
(array([346]), array([480]))
0.8999999761581421
Time to process polygon= 0.6004166603088379
(array([378]), array([295]))
0.0
Time to process polygon= 0.6017782688140869
(array([365, 365, 366]), array([110, 111, 111]))
0.0
Time to process polygon= 0.7660911083221436
(array([351, 351]), array([221, 222]))
0.0
Time to process polygon= 0.6833028793334961
(array([599, 599, 600, 600]), array([142, 1

(array([659]), array([212]))
0.0
Time to process polygon= 0.5271553993225098
(array([309, 309, 310, 310, 311, 311]), array([243, 244, 243, 244, 243, 244]))
0.0
Time to process polygon= 0.6981015205383301
(array([436, 436, 437, 437]), array([93, 94, 93, 94]))
0.0
Time to process polygon= 0.7765054702758789
(array([594, 594, 594, 595, 595]), array([177, 178, 179, 177, 178]))
0.0
Time to process polygon= 0.6126065254211426
(array([569, 569, 570, 570]), array([217, 218, 217, 218]))
0.0
Time to process polygon= 0.5873720645904541
(array([337]), array([360]))
0.0
Time to process polygon= 0.5829215049743652
(array([351, 351, 352, 352, 352, 353]), array([240, 241, 239, 240, 241, 240]))
0.0
Time to process polygon= 0.6936259269714355
(array([278]), array([232]))
0.0
Time to process polygon= 0.7269887924194336
(array([347, 347, 348, 348]), array([239, 240, 239, 240]))
0.0
Time to process polygon= 0.6819744110107422
(array([617]), array([234]))
0.0
Time to process polygon= 0.5271730422973633
(arr

(array([575, 576]), array([155, 155]))
3.814395388974325
Time to process polygon= 0.6192541122436523
2019-08-23
2019 08 23
/data2/lthapa/2019/STAGEIV/ST4.2019082312.24h.nc
(array([589]), array([231]))
0.0
Time to process polygon= 0.5465235710144043
(array([644, 644]), array([258, 259]))
1.8043632067040185
Time to process polygon= 0.5113236904144287
(array([431]), array([271]))
0.0
Time to process polygon= 0.6011161804199219
(array([670]), array([135]))
0.0
Time to process polygon= 0.5871908664703369
(array([683, 684]), array([141, 141]))
0.0
Time to process polygon= 0.5732369422912598
(array([378, 378, 379]), array([295, 296, 296]))
0.0
Time to process polygon= 0.6157169342041016
(array([365, 366]), array([111, 111]))
0.0
Time to process polygon= 0.7652039527893066
(array([351, 351]), array([221, 222]))
0.0
Time to process polygon= 0.6831908226013184
(array([599, 599, 600, 600]), array([142, 143, 142, 143]))
0.0
Time to process polygon= 0.6222052574157715


In [11]:
#fire_features[fire_features['Current Day']=='2020-07-27']
therest = fire_features[['Incident Number', 'Fire Name', 'Current Day','Lat Fire', 'Lon Fire','Number of VIIRS points', 'A_PCP_GDS5_SFC_acc24h_0', 'APCP_P8_L1_GST0_acc_0','A_PCP_GDS5_SFC_acc24h_1', 'APCP_P8_L1_GST0_acc_1' ]]
therest

#therest.to_csv('fire_features_precip_2019.csv', index=False)
#therest.to_csv('fire_features_precip_2020.csv', index=False)
therest.to_csv('fire_features_precip_1dayout.csv', index=False)

## Helper Functions

In [7]:
#this function calculates the area of each grid in a given grid cell
# LAT and LON should be 2d
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LAT[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    lon_corners = (LON[0:(lat.shape[0]-1),  0:(lat.shape[1])-1] + LON[1:(lat.shape[0]), 1:(lat.shape[1])])/2
    return lat_corners, lon_corners
    """for ii in range(1,LAT.shape[0]-1):
        for jj in range(1, LAT.shape[1]-1):
            cell_lat = LAT[ii, jj]
            cell_lon = LON[ii, jj]
            SW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            SE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii-1,jj-1] - LAT[ii,jj])/2
            SE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
            NW_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NW_lon[ii,jj] = LON[ii,jj] + (LON[ii-1,jj-1] - LON[ii,jj])/2
            NE_lat[ii,jj] = LAT[ii,jj] + (LAT[ii,jj] - LAT[ii-1,jj-1])/2
            NE_lon[ii,jj] = LON[ii,jj] + (LON[ii,jj] - LON[ii-1,jj-1])/2
    return  SW_lat, SW_lon, SE_lat, SE_lon, NW_lat, NW_lon, NE_lat, NE_lon"""

In [8]:
# this function calculates the overlap between the given shape (fire_shape) and given grid (LAT, LON). 
#LAT and LON should be 2D

def calculate_overlaps(lat, lon, lat_corners, lon_corners, lat_start, lat_stop, lon_start, lon_stop, fire_shape):
    overlaps = np.zeros(lat.shape)
    # buffer the fire shape so it has a non zero area
    if fire_shape.area ==0:
        fire_shape = fire_shape.buffer(0.01)
    # find the intersection between the first day's polygon and the  grid
    for ii in range(lat_start, lat_stop):
        for jj in range(lon_start, lon_stop):
            #print('ctr', lat[ii,jj], lon[ii,jj])
            """ SW_lat = lat_corners[ii-1, jj-1]
            SW_lon = lon_corners[ii-1, jj-1]
            SE_lat = lat_corners[ii-1, jj-1]
            SE_lon = lon_corners[ii-1, jj]
            NW_lat = lat_corners[ii, jj-1]
            NW_lon = lon_corners[ii, jj-1]
            NE_lat = lat_corners[ii, jj]
            NE_lon = lon_corners[ii, jj]"""
            #print((lon_corners[ii, jj-1],lat_corners[ii, jj-1]), (lon_corners[ii, jj],lat_corners[ii, jj]), (lon_corners[ii-1, jj],lat_corners[ii-1, jj-1]), (lon_corners[ii-1, jj-1], lat_corners[ii-1, jj-1]))
            poly_cell = Polygon([(lon_corners[ii, jj-1],lat_corners[ii, jj-1]), (lon_corners[ii, jj],lat_corners[ii, jj]), (lon_corners[ii-1, jj],lat_corners[ii-1, jj-1]), (lon_corners[ii-1, jj-1], lat_corners[ii-1, jj-1])])
            overlap_cell = fire_shape.intersection(poly_cell)
            #overlaps are the fraction of the POLYGON in each grid cell
            overlaps[ii,jj] = overlap_cell.area/fire_shape.area
    
    return overlaps

In [ ]:
#geoseries way
        #load in the relevant geometry array
        geojson_path = path.replace('.'+yr+mo+dy+'.nc4', '_grid_polygons.geojson')
        geojson_path = geojson_path.replace('/data2/lthapa/'+yr+'/MERRA2/', './')
        geojson_path = geojson_path.replace('/data2/lthapa/'+yr+'/QFED/', './')
        print(geojson_path)
        grid_polys = gpd.read_file(geojson_path)
        
inds = np.where(grid_polys['geometry'].intersection(poly).area)[0]
            areas = grid_polys['geometry'].intersection(poly).area
            #print(areas[inds])
            #print(areas[inds]/poly.area)
            toc=time.time()
            print('Time elapsed for calculating overlaps for polygon:', toc-tic)


In [9]:
#fnction is from here: https://gis.stackexchange.com/questions/71630/subsetting-a-curvilinear-netcdf-file-roms-model-output-using-a-lon-lat-boundin
def bbox2ij(lon,lat,bbox):
    bbox=np.array(bbox)
    mypath=np.array([bbox[[0,1,1,0]],bbox[[2,2,3,3]]]).T
    p = path.Path(mypath)
    points = np.vstack((lon.flatten(),lat.flatten())).T
    n,m = np.shape(lon)
    inside = p.contains_points(points).reshape((n,m))
    ii,jj = np.meshgrid(range(m),range(n)) #ii is the columns, jj is the rows
    return min(ii[inside]),max(ii[inside]),min(jj[inside]),max(jj[inside])